Эту задачу можно решить контентной или совместной фильтрацией.
Первая использует сходство между элементами, чтобы рекомендовать элементы, похожие на то, что нравится пользователю.

Вторая одновременно использует сходство между запросами и элементами для предоставления рекомендаций.


Меры подобия:
- Косинус
- скалярное произведение
- Евклидово расстояние



Этапы:
- Candidate Generation
- Scoring
- Re-ranking

Техники, которые стоит использовать:
- TD-IDF Vectorizer на описания фильмов для подсчета их похожести
- Брать только первых 3 актёров, так как в основном остается впечатление по главным ролям (не полностью, естественно, но в данной задаче второстепенными ролями можно пренебречь)
- Сделать для датасета фильмов новую фичу, куда внести режиссера, актеров, ключевые слова и жанр. На эту фичу также применить TD_IDF Vectorizer.
- Каким-то образом придать больший вес режиссеру.
- Для фильмов факт полного просмотра играет большую роль, а для сериалов - нет. Для них можно сделать какой-то порог процента просмотра.

In [ ]:
!unzip "/content/kion_data.zip" -d "/content/kion_data"

Archive:  /content/kion_data.zip
   creating: /content/kion_data/kion_data/
  inflating: /content/kion_data/__MACOSX/._kion_data  
  inflating: /content/kion_data/kion_data/.DS_Store  
  inflating: /content/kion_data/__MACOSX/kion_data/._.DS_Store  
  inflating: /content/kion_data/kion_data/users.csv  
  inflating: /content/kion_data/kion_data/train_data.csv  
  inflating: /content/kion_data/kion_data/items.csv  
  inflating: /content/kion_data/kion_data/test_data.csv  


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 22.3 MB/s eta 0:00:00


In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
data_folder = "kion_data/kion_data/"

users_df = pd.read_csv(data_folder + "users.csv")
items_df = pd.read_csv(data_folder + "items.csv")
train_part = pd.read_csv(data_folder + "train_data.csv", parse_dates=["last_watch_dt"])
test_part = pd.read_csv(data_folder + "test_data.csv")
test_part = test_part.groupby("user_id").agg({"ground_truth": list}).reset_index()

In [ ]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,721985,age_45_54,income_20_40,Ж,0
3,704055,age_35_44,income_60_90,Ж,0
4,846063,age_35_44,income_40_60,Ж,1


In [ ]:
items_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,11114,film,Принцесса Лебедь: Пират или принцесса,"The Swan Princess: Princess Tomorrow, Pirate T...",2016.0,"для детей, сказки, полнометражные, зарубежные,...",США,NaN,6.0,Sony Pictures,Ричард Рич,"Брайан Ниссен, Гарднер Джаэс, Грант Дураззо, Д...",Анимационная сказка о непоседливой принцессе Э...,"Принцесса, Лебедь, Пират, или, принцесса, 2016..."


In [ ]:
train_part.head(100)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0
...,...,...,...,...,...
95,470881,11800,2021-06-21,909.0,19.0
96,231634,2301,2021-07-06,51429.0,100.0
97,370816,14071,2021-04-19,34.0,1.0
98,17110,4740,2021-07-08,40.0,1.0


38.0

In [ ]:
items_df[items_df['item_id'] == 11800]["content_type"] == "film"


3563    True
Name: content_type, dtype: bool

In [ ]:
import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

income_chart = categorical_histogram(users_df, *['income'], **{})
income_chart

In [ ]:
age_chart = categorical_histogram(users_df, *['age'], **{})
age_chart

In [ ]:
items_df["actors"]

0        Адольфо Фернандес, Ана Фернандес, Дарио Гранди...
1        Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...
2        Адриан Холмс, Даррен Шалави, Джерри Вассерман,...
3        Александра Риддлстон-Барретт, Джеральдин Джейм...
4        Брайан Ниссен, Гарднер Джаэс, Грант Дураззо, Д...
                               ...                        
12062    Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...
12063    Виктория Исакова, Александр Кузьмин, Алексей М...
12064    Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...
12065    Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...
12066                   Мкртыч Арзуманян, Виктория Рунцова
Name: actors, Length: 12067, dtype: object

In [ ]:
# ACHTUNG! DO NOT TOUCH

def ndcg_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)
    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_


def dcg(scores: np.ndarray) -> float:
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )


def recall_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    n_gt = len(gt_items)
    intersection = len(set(gt_items).intersection(set(predicted)))
    return intersection / n_gt


def evaluate_recommender(df: pd.DataFrame, model_preds_col: str, gt_col: str = "ground_truth") -> Dict[str, float]:
    metric_values = []

    for _, row in df.iterrows():
        metric_values.append(
            (ndcg_metric(row[gt_col], row[model_preds_col]), recall_metric(row[gt_col], row[model_preds_col]))
        )

    return {"ndcg": np.mean([x[0] for x in metric_values]), "recall": np.mean([x[1] for x in metric_values])}

In [ ]:
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pd.DataFrame) -> None:
        # реализация может быть любой, никаких ограничений

        # не забудьте про
        self.trained = True

    @abstractmethod
    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращать список массивов из item_id, которые есть в `item_df`, чтобы корректно работал подсчет метрик
        pass

In [ ]:
class TopPopular(BaseRecommender):
    def __init__(self):
        super().__init__()

    def fit(self, df: pd.DataFrame, item_id_col: str = "item_id") -> None:
        # считаем популярность айтемов
        self.recommendations = df[item_id_col].value_counts().index.values
        self.trained = True

    def predict(self, df: pd.DataFrame, topn: int = 10) -> np.ndarray:
        assert self.trained
        # возвращаем для всех одно и то же
        return [self.recommendations[:topn]] * len(df)


toppop = TopPopular()
toppop.fit(train_part)
test_part["toppopular_recs"] = toppop.predict(test_part)
test_part.head()

,user_id,ground_truth,toppopular_recs
0,30,"[8584, 3031, 15363]","[10440, 15297, 13865, 9728, 4151, 2657, 3734, ..."
1,144,"[3800, 11987, 10942]","[10440, 15297, 13865, 9728, 4151, 2657, 3734, ..."
2,241,"[14841, 6162, 5808, 15266]","[10440, 15297, 13865, 9728, 4151, 2657, 3734, ..."
3,259,"[16509, 10509, 4491, 5434, 4471, 10772, 817]","[10440, 15297, 13865, 9728, 4151, 2657, 3734, ..."
4,475,"[6214, 3734, 6686]","[10440, 15297, 13865, 9728, 4151, 2657, 3734, ..."


In [ ]:
evaluate_recommender(df=test_part, model_preds_col="toppopular_recs")

{'ndcg': 0.17037237918248196, 'recall': 0.0763696799665908}

Суть контентно‑ориентированного метода — в сопоставлении пользователей с теми товарами или контентом, которые им нравились или были куплены ранее. Такие системы подходят, например, для рекомендаций фильмов. Они учитывают такие параметры, как режиссер, актеры, жанр и ищут сходство с другими лентами. Контентно‑ориентированный подход достаточно простой и хорошо зарекомендовал себя, но его эффективность зависит от предметной области. Он плохо подходит при покупке бытовой техники или мебели.

Что нужно сделать:
1. Для колларобативной фильтрации определить, как понять, понравился ли фильм или сериал пользователю, какие факторы стоит при этом учесть. Может, использовать при этом веса ?
2. Построить матрицу для коллаборативной фильтрации
3. Рекомендовать фильмы/сериалы на этой основе

Для контентной фильтрации


# Использованные материалы
- https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada
